In [1]:
import gym
import rlbench.gym
from stable_baselines3 import A2C
from stable_baselines3.a2c.policies import MlpPolicy, CnnPolicy

In [2]:
env_name = 'reach_target-state-v0'
render_mode = 'human'
observation_mode = 'state'
policy_mode = 'MlpPolicy'
model_name = 'A2C'



In [4]:
class myEnv(gym.Wrapper):
    def __init__(self, env=None, max_steps_episode = 500):
        super(myEnv, self).__init__(env)
        self._step_counter = 0
        self._max_steps_episode = max_steps_episode
        
    def step(self, action):
        self._step_counter += 1
        obs, reward, done, info = self.env.step(action)
        if(self._step_counter > self._max_steps_episode):
            done = True
            reward = -1
            self._step_counter = 0
            print("Maximum steps reacheda tardado mucho")
        return obs, reward, done, info


In [5]:
name_model = env_name + '_' + render_mode + '_' + model_name
env = gym.make(env_name, render_mode=render_mode, observation_mode=observation_mode)


In [6]:
env = myEnv(env)


In [7]:
model = A2C(policy_mode, env, verbose=1, tensorboard_log="./tb_log_name/")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
model.learn(300, eval_freq=100, n_eval_episodes=5, eval_log_path="./logs/")

Logging to ./tb_log_name/A2C_4
a cascarla, ha tardado mucho


In [21]:
#print(env.observation_space)
vars(env.env._robot.arm)
#print(env.env._robot.arm.__dict__)

{'_handle': 12,
 '_num_joints': 7,
 'joints': [<pyrep.objects.joint.Joint at 0x7fdde49bc280>,
 '_joint_handles': [13, 15, 17, 19, 21, 23, 25],
 'max_velocity': 1.0,
 'max_acceleration': 4.0,
 'max_jerk': 1000,
 '_ik_target': <pyrep.objects.dummy.Dummy at 0x7fdde51fe5e0>,
 '_ik_tip': <pyrep.objects.dummy.Dummy at 0x7fdde5204760>,
 '_ik_group': 2030003,
 '_collision_collection': 2000000}

In [14]:
obs = env.reset()

In [43]:
vars(env.task)

{'_pyrep': <pyrep.pyrep.PyRep at 0x7fdd283ef190>,
 '_robot': <rlbench.backend.robot.Robot at 0x7fdd283ef940>,
 '_scene': <rlbench.backend.scene.Scene at 0x7fdd283ef9a0>,
 '_task': <rlbench.tasks.reach_target.ReachTarget at 0x7fdd283f8070>,
 '_variation_number': 0,
 '_action_mode': <rlbench.action_modes.ActionMode at 0x7fdd283ef100>,
 '_dataset_root': '',
 '_obs_config': <rlbench.observation_config.ObservationConfig at 0x7fdd28444ca0>,
 '_static_positions': False,
 '_attach_grasped_objects': True,
 '_reset_called': True,
 '_prev_ee_velocity': None,
 '_enable_path_observations': False,
 '_robot_shapes': [<pyrep.objects.shape.Shape at 0x7fdd283f8ca0>,
  <pyrep.objects.shape.Shape at 0x7fdd284002b0>]}

In [54]:
import numpy as np
tip = env.env._robot.arm.get_tip()
print(obs)
velocities = env.env._robot.arm.get_joint_velocities()
positions = env.env._robot.arm.get_joint_positions()
forces = env.env._robot.arm.get_joint_forces()
gripper_open = 1.0 if env.env._robot.gripper.get_open_amount()[0] > 0.9 else 0.0
gripper_pose = np.array(tip.get_pose())
gripper_matrix = tip.get_matrix()
gripper_touch_forces= env.env._robot.gripper.get_touch_sensor_forces()
gripper_joint_positions = np.array(env.env._robot.gripper.get_joint_positions())
task_low_dim_state = env.task._task.get_low_dim_state()


[ 1.00000000e+00  3.33786011e-05 -2.11238861e-03 -4.76837158e-06
  1.99317932e-03 -1.47819519e-04  7.29560852e-04 -1.14440918e-04
 -7.14897942e-06  1.75480872e-01 -8.55001781e-06 -8.73202741e-01
 -2.25986332e-06  1.22135472e+00  7.85387397e-01 -2.10542828e-02
  2.39683132e+01 -3.96758839e-02 -1.62904129e+01 -1.40298635e-01
 -4.38615799e+00  3.43917273e-02  2.78630733e-01 -8.15699063e-03
  1.47156966e+00 -2.37252493e-06  9.92713988e-01 -2.94828806e-06
  1.20494887e-01  3.99997458e-02  3.99982594e-02  1.79127906e-03
 -4.32483852e-04  1.14887951e-06  1.78757461e-03  4.32065135e-04
 -7.01844669e-07  3.89102101e-02  6.76660985e-02  1.18909001e+00]


In [55]:
print(gripper_open, velocities, positions, forces, gripper_pose, gripper_joint_positions, gripper_touch_forces, task_low_dim_state)
print(gripper_pose)

1.0 [3.337860107421875e-05, -0.0021123886108398438, -4.76837158203125e-06, 0.0019931793212890625, -0.00014781951904296875, 0.0007295608520507812, -0.00011444091796875] [-7.1489794208901e-06, 0.17548087239265442, -8.550017810193822e-06, -0.8732027411460876, -2.2598633222514763e-06, 1.2213547229766846, 0.785387396812439] [-0.021054282784461975, 23.968313217163086, -0.03967588394880295, -16.29041290283203, -0.14029863476753235, -4.386157989501953, 0.034391727298498154] [ 2.78630733e-01 -8.15699063e-03  1.47156966e+00 -2.37252493e-06
  9.92713988e-01 -2.94828806e-06  1.20494887e-01] [0.03999975 0.03999826] [[0.0017912790644913912, -0.00043248385190963745, 1.1488795053082868e-06], [0.0017875746125355363, 0.0004320651351008564, -7.018446694928571e-07]] [0.03891021 0.0676661  1.18909001]
[ 2.78630733e-01 -8.15699063e-03  1.47156966e+00 -2.37252493e-06
  9.92713988e-01 -2.94828806e-06  1.20494887e-01]


In [64]:
dir(env.env._robot.arm)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_assert_len',
 '_check_model',
 '_collision_collection',
 '_get_model_property',
 '_get_objects_in_tree',
 '_get_property',
 '_get_requested_type',
 '_handle',
 '_ik_group',
 '_ik_target',
 '_ik_tip',
 '_joint_handles',
 '_num_joints',
 '_set_model_property',
 '_set_property',
 'check_arm_collision',
 'check_collision',
 'check_distance',
 'copy',
 'exists',
 'get_bounding_box',
 'get_bullet_friction',
 'get_configs_for_tip_pose',
 'get_configuration_tree',
 'get_contact',
 'get_explicit_handling',
 'get_extension_string',
 'get_handle',
 'get_jacobian',
 'get_joint_count',
 'get_joint_forces',
 'get_joint_intervals',
 '

In [5]:
picture = env.render()
from matplotlib import pyplot as PLT
PLT.show(picture)

In [7]:
print(picture)

None


In [8]:
env.render()

In [9]:
a=env.render(mode=render_mode)

In [10]:
a

In [11]:
obs = env.reset()

In [12]:
img = obs['front_rgb']
plt.imshow(img)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [13]:
print(obs)

[ 1.00000000e+00  9.53674316e-06  2.57492065e-04  1.71661377e-04
 -1.05857849e-03 -5.24520874e-05 -1.11579895e-03  1.14440918e-04
 -2.38060784e-06  1.75389558e-01  8.85453755e-06 -8.73069942e-01
  1.10915771e-05  1.22126341e+00  7.85393119e-01 -2.34539504e-03
  2.35156212e+01 -3.39262970e-02 -1.59582062e+01 -1.56218112e-01
 -4.27472019e+00  3.19468454e-02  2.78617293e-01 -8.14859569e-03
  1.47162831e+00 -8.86944144e-06  9.92707789e-01  1.32834577e-06
  1.20545931e-01  3.99983749e-02  3.99983153e-02  1.77217578e-03
 -4.39256430e-04  8.94069672e-08  1.76891091e-03  4.38576943e-04
  2.14576716e-07  2.55182534e-01 -2.56085605e-03  1.16568923e+00]


In [1]:
fig.__dict__

NameError: name 'fig' is not defined